<a href="https://colab.research.google.com/github/jaycrossler/ai-training/blob/main/Ollama%20and%20pydantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Outputs from pydantic-ai



## Step 1: Create a LLM server with ollama

To run this notebook, we need to have a OpenAI Compatible server. You can connect you own OpenAI account, huggingface CLI or use a local server. In the next cell, we will create an LLM server running on colab so that you dont' need to use any of the prior options.
> Note: If you are running this code on the Google Colab, be sure to check if you have a GPU (Runtime menu->`Change runtime type`->`gpu T4`).

In [25]:
# Download and install Ollama which will serve the LLM
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [26]:
# Importing nesseracy libraries
import subprocess
import time

In [63]:
# Start ollama in the background and use llama3.1 model

# Start the process in the background
server = subprocess.Popen(['ollama', 'serve'])
time.sleep(60) # To make sure ollama is ready in subsequent cell if you are running all not cell at a time

# To kill the server
# server.kill()

# To see all the models available: https://ollama.com/library
# Note: llama3.3 takes about 10 minutes to install and is slow to run (need more ram)
# Note: phi-4 and deepseek don't support tools in the pydantic format, so 3.2 seems best
MODEL = 'llama3.2'
llama3 = subprocess.Popen(['ollama', 'run', MODEL])
#time.sleep(90) # Make sure ollama is ready in subsequent cell if you are running all not cell at a time

# To kill the llama3
# llama3.kill()

In [50]:
subprocess.Popen(['ollama', 'run', 'phi4'])

<Popen: returncode: None args: ['ollama', 'run', 'phi4']>

In [69]:
# show which model(s) ollama is serving
!ollama list

NAME                  ID              SIZE      MODIFIED          
phi4:latest           ac896e5b8b34    9.1 GB    2 minutes ago        
llama3.3:latest       a6eb4748fd29    42 GB     2 minutes ago        
deepseek-r1:latest    0a8c26691023    4.7 GB    3 minutes ago        
llama3.2:latest       a80c4f17acd5    2.0 GB    11 minutes ago       
llama3.1:latest       46e0c10c039e    4.9 GB    About an hour ago    


In [29]:
!pip install pydantic-ai


In [43]:
from pydantic_ai import Agent

agent = Agent(
    'ollama:llama3.2'
    #system_prompt='Be concise, reply with one sentence.',
)

result = await agent.run('Where were the olympics held in 2012 ?')
print(result.data)

The Olympics held in 2012 were actually two separate events: the Winter Olympics weren't held in 2012, but that's a story for another time.

I'm assuming you're referring to the Summer Olympics. The 2012 Summer Olympics, formally known as Games of the XXX Olympiad, took place from July 27 to August 12, 2012. These Olympic Games were held in London, United Kingdom.


In [72]:
from dataclasses import dataclass

from pydantic import BaseModel, Field

from pydantic_ai import Agent, RunContext, UnexpectedModelBehavior


class DatabaseConn:
    """This is a fake database for example purposes.

    In reality, you'd be connecting to an external database
    (e.g. PostgreSQL) to get information about customers.
    """

    @classmethod
    async def customer_name(cls, *, id: int) -> str | None:
        if id == 123:
            return 'John'

    @classmethod
    async def customer_balance(cls, *, id: int, include_pending: bool) -> float:
        if id == 123:
            return 123.45
        else:
            raise ValueError('Customer not found')


@dataclass
class SupportDependencies:
    customer_id: int
    db: DatabaseConn


class SupportResult(BaseModel):
    support_advice: str = Field(description='Advice returned to the customer')
    block_card: bool = Field(description='Whether to block their card')
    risk: int = Field(description='Risk level of query', ge=0, le=10)


support_agent = Agent(
    'ollama:llama3.2',
    deps_type=SupportDependencies,
    result_type=SupportResult,
    retries=20,
    system_prompt=(
        'You are a support agent in our bank, give the '
        'customer support and judge the risk level of their query. '
        "Reply using the customer's name."
    ),
)


@support_agent.system_prompt
async def add_customer_name(ctx: RunContext[SupportDependencies]) -> str:
    customer_name = await ctx.deps.db.customer_name(id=ctx.deps.customer_id)
    return f"The customer's name is {customer_name!r}"


@support_agent.tool
async def customer_balance(
    ctx: RunContext[SupportDependencies], include_pending: bool
) -> str:
    """Returns the customer's current account balance."""
    balance = await ctx.deps.db.customer_balance(
        id=ctx.deps.customer_id,
        include_pending=include_pending,
    )
    return f'${balance:.2f}'


if __name__ == '__main__':
    deps = SupportDependencies(customer_id=123, db=DatabaseConn())
    try:
      result = await support_agent.run('What is my name and balance?', deps=deps)
      print(result.data)
    except UnexpectedModelBehavior:
      print('Error')
      raise

    try:
      result = await support_agent.run('I just lost my card!', deps=deps)
      print(result.data)
    except UnexpectedModelBehavior:
      print('Error')
      raise


Error


UnexpectedModelBehavior: Exceeded maximum retries (10) for result validation